# GSM-EVP-LIH

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_EVP
gsm = GSM1D_EVP()
display(Markdown(gsm.latex_potentials()))

## Monotonic strain-driven loading

In [ ]:
mparams = dict(E = 20, K = 5, S = 2, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=100)
n_t = 1510
#n_t = 11
n_I = 1
eps_max = 5.0
t_t = np.linspace(0, 10, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
eps_t = eps_ta[:, 0]
sig_t = sig_ta[:, 0]
eps_p_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Z_t = Sig_t[:, 0, :].T
sig_t_strain = np.copy(sig_t)[:, 0]
sig_t_strain.shape

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()

ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Stress-driven monotonic loading

In [ ]:
response = gsm.get_G_response(sig_t_strain, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
iter_t = iter_t[:, 0]
eps_t = eps_ta[:, 0, 0]
sig_t = sig_ta[:, 0]
eps_p_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Z_t = Sig_t[:, 0, :].T
eps_t.shape, sig_t.shape

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, sig_t, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Cyclic strain-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
final_time = 1
eps_max = 2
n_t = 151
#n_t = 31
t_t = np.linspace(0, final_time, n_t)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*final_time, 0.5*final_time, 100*final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
mparams = dict(E = 20, K = 5, S = 0.2, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=1)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_p_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ((ax_sig, ax_eps), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()

ax_sig.plot(t_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
# ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(t_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()